In [ ]:
from tifffile import imread, imsave
from stardist.matching import  matching
import numpy as np
from pathlib import Path
from skimage.segmentation import relabel_sequential

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def color_cells(volume, idcs):
    """
    background = 0
    not in idcs = 1
    in idcs = 2
    """
    idcs = np.asarray(idcs)
    lut = np.ones(np.max(volume)+1)
    lut[idcs] = 2
    lut[0] = 0
    return lut[volume]

In [ ]:
def create_fn_fp_tifs(output_fn, output_fp, prediction_path, ground_truth_path, thresh=0.5, remove_pred_slice=False):
    
    Y_pred_paths, Y_true_paths = prediction_path, ground_truth_path
    
    if remove_pred_slice:
        Y_pred = imread(str(Y_pred_paths))
        Y_true = imread(str(Y_true_paths))[1:]
    else:
        Y_pred, Y_true = tuple(imread(str(paths)) for paths in [Y_pred_paths, Y_true_paths]) 
    Y_pred, Y_true = tuple(relabel_sequential(Y_)[0] for Y_ in [Y_pred, Y_true])

    m = matching(Y_true, Y_pred, thresh=0.5, report_matches=True)

    matching_dict = m._asdict()
    matched_pairs = np.asarray(matching_dict['matched_pairs'])
    matched_tps = np.asarray(matching_dict['matched_tps'])
    
    Y_true_ids = matched_pairs[matched_tps, 0]
    Y_pred_ids = matched_pairs[matched_tps, 1]

    Y_true_, Y_pred_ = color_cells(Y_true, Y_true_ids), color_cells(Y_pred, Y_pred_ids)
    
    imsave(output_fn, Y_true_, compress=9)
    imsave(output_fp, Y_pred_, compress=9)

In [ ]:
# biofilmQ + Huygens output

In [ ]:
gt_path_mask = '../../chapter_single_cell_segmentation/full_semimanual-huy/test/masks/im0.tif'
prediction= '../../chapter_single_cell_segmentation/full_semimanual-huy/test/images_Pos1/2020-08-17_data_seeded_watershed/huy_seeded_watershed.tif'

In [ ]:
create_fn_fp_tifs('biofilmQ_fn.tif', 'biofilmQ_fp.tif', prediction, gt_path_mask)

In [ ]:
# 'stardist_192_48x96x96_patches-semimanual-raw-64x128x128_True_70prc_rep5' # best network

In [ ]:
prediction = '../predictions/stardist_192_48x96x96_patches-semimanual-raw-64x128x128_True_70prc_rep5/datasets/full_semimanual-raw/test/images/im0.tif'

In [ ]:
create_fn_fp_tifs('stardist_fn.tif', 'stardist_fp.tif', prediction, gt_path_mask, remove_pred_slice=True)

In [ ]:
# now use the matlab script prepare2_tiff2vtk.m 

In [ ]:
!pwd